In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB


we want to import the train and test data but will only be working on the train data

In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
total_data = pd.concat([test_data.assign(ind="test"), train_data.assign(ind="train")])

DATA EXPLORATION AND ANALYSIS


It is necessary we look at the information about each column

Next, we take a look at the missing values and also what percent of data point is missing in our columns

In [3]:
total_data.columns = ["id", "p_class", "name", "sex", "age", "sibling_spouse", "parch", "ticket", "fare", "cabin", "embarked", "ind", "survived"]

In [4]:
missing_data = pd.DataFrame({"sum_missing" : train_data.isna().sum(),
                "percent missing" : train_data.isna().sum()/891 * 100 })
missing_data

,sum_missing,percent missing
PassengerId,0,0.000000
Survived,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,177,19.865320
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,0,0.000000


In [5]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


We can see that there are 177 missing age values, 687 missing cabin values and 2 missing age values.

DATA CLEANING

It is necessary to create new features so that we can be able to get insights from our data. we converted some numerical features to category and also categorical features to numericals

In [6]:
sex = {"male": 0, "female": 1}
total_data  ["no_of_relatives"] = total_data["sibling_spouse"] + total_data["parch"]
total_data["sex_cat"] = total_data["sex"].map(sex)


In [7]:
total_data.age = total_data.age.fillna(0)


In [8]:
def relative(x):
    if x > 0:
        return 1
    else:
        return 0
     
    
total_data["not_alone"] = total_data["no_of_relatives"].apply(relative)


In [9]:
encoder = OneHotEncoder(handle_unknown="ignore")
encoder_embark = pd.DataFrame(encoder.fit_transform(total_data[["embarked"]]).toarray())
join_embark = total_data.join(encoder_embark)

In [10]:
total_data.fare = pd.cut(total_data.fare,
                         bins = [0., 8.0, 14.0, 31.0, 99.0, 250.0, 400.0, np.inf],
                         labels=[1, 2, 3, 4, 5, 6, 7])

In [11]:
total_data.drop(["ticket", "id", "name", "sex", "cabin", "embarked", "age"], axis = 1, inplace = True)

In [12]:
test,train = total_data[total_data["ind"].eq("test")], total_data[total_data["ind"].eq("train")] 
train.survived.isna().sum()
# train["survived"] = train["survived"].fillna(0)
# test["survived"] = test["survived"].fillna(0)

0

In [13]:
train_data = train.drop(["ind"], axis = 1)
test_data = test.drop(["ind"], axis = 1)
X = train_data.drop(["survived"], axis = 1)
y = test_data.survived
X_train, y_train = train_test_split(train_data, test_size= 0.3)
X_test, y_test = train_test_split(test_data, test_size=0.3)

In [14]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

Y_pred = logreg.predict(X_test)

acc_log = round(logreg.score(X_train, y_train) * 100, 2)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
sgd = SGDClassifier(max_iter=5, tol=None)
sgd.fit(X_train, y_train)
y_predict = sgd.predict(X_test)

sgd.score(X_train, y_train)
acc_sgd = round(sgd.score(X_train, y_train) * 100, 2)

ValueError: y should be a 1d array, got an array of shape (268, 8) instead.